In [ ]:
from google.colab import files
files.upload()

In [ ]:
from keras.models import load_model
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

In [ ]:
ridingModel = load_model('/content/Riding.h5')
crackingModel = load_model('/content/Cracking.h5')
ruttingModel = load_model('/content/Rutting.h5')

In [ ]:
ridingdf = pd.read_excel('2.xlsx',sheet_name=0)
crackingdf = pd.read_excel('2.xlsx',sheet_name=1)
ruttingdf = pd.read_excel('2.xlsx',sheet_name=2)

In [ ]:
xriding = ridingdf.iloc[:,0:8].values
yriding = np.reshape(ridingdf.iloc[:,8].values, newshape=(-1, 1))
xcracking = crackingdf.iloc[:,0:8].values
ycracking = np.reshape(crackingdf.iloc[:,8].values, newshape=(-1, 1))
xrutting = ruttingdf.iloc[:,0:8].values
yrutting = np.reshape(ruttingdf.iloc[:,8].values, newshape=(-1, 1))

In [ ]:
# scalers
scaler_x_riding = StandardScaler()
scaler_y_riding = StandardScaler()
scaler_x_cracking = StandardScaler()
scaler_y_cracking = StandardScaler()
scaler_x_rutting = StandardScaler()
scaler_y_rutting = StandardScaler()
# scaling riding
x_train_riding_sc = scaler_x_riding.fit_transform(xriding)
y_train_riding_sc = scaler_y_riding.fit_transform(yriding)
# scaling cracking
x_train_cracking_sc = scaler_x_cracking.fit_transform(xcracking)
y_train_cracking_sc = scaler_y_cracking.fit_transform(ycracking)
# scaling rutting
x_train_rutting_sc = scaler_x_rutting.fit_transform(xrutting)
y_train_rutting_sc = scaler_y_rutting.fit_transform(yrutting)

In [ ]:
# prediction Riding
train_riding_predict = [scaler_y_riding.inverse_transform(ridingModel.predict(x_train_riding_sc))[i][0] for i in range(x_train_riding_sc.shape[0])]
train_riding_act = [scaler_y_riding.inverse_transform(y_train_riding_sc)[i][0] for i in range(y_train_riding_sc.shape[0])]
meantrainriding =[abs(train_riding_act[i]-train_riding_predict[i])/train_riding_act[i] for i in range(len(train_riding_act))]
# prediction cracking
train_cracking_predict = [scaler_y_cracking.inverse_transform(crackingModel.predict(x_train_cracking_sc))[i][0] for i in range(x_train_cracking_sc.shape[0])]
train_cracking_act = [scaler_y_cracking.inverse_transform(y_train_cracking_sc)[i][0] for i in range(y_train_cracking_sc.shape[0])]
meantraincracking =[abs(train_cracking_act[i]-train_cracking_predict[i])/train_cracking_act[i] for i in range(len(train_cracking_act))]
# prediction rutting
train_rutting_predict = [scaler_y_rutting.inverse_transform(ruttingModel.predict(x_train_rutting_sc))[i][0] for i in range(x_train_rutting_sc.shape[0])]
train_rutting_act = [scaler_y_rutting.inverse_transform(y_train_rutting_sc)[i][0] for i in range(y_train_rutting_sc.shape[0])]
meantrainrutting =[abs(train_rutting_act[i]-train_rutting_predict[i])/train_rutting_act[i] for i in range(len(train_rutting_act))]


In [ ]:
Train_PCI_predict= [(0.4*train_riding_predict[i])+(0.4*train_cracking_predict[i])+(0.2*train_rutting_predict[i]) for i in range(len(train_riding_predict))] 
Train_PCI_act = [(0.4*train_riding_act[i])+(0.4*train_cracking_act[i])+(0.2*train_rutting_act[i]) for i in range(len(train_riding_predict))]
meantrainPCI= [abs(Train_PCI_act[i]-Train_PCI_predict[i])/Train_PCI_act[i] for i in range(len(Train_PCI_predict))]

In [ ]:
print(f"mean absolute error for train Riding is {round(np.mean(meantrainriding)*100,ndigits=1)}%")
print(f"mean absolute error for train cracking is {round(np.mean(meantraincracking)*100,ndigits=1)}%")
print(f"mean absolute error for train rutting is {round(np.mean(meantrainrutting)*100,ndigits=1)}%")
print(f"mean absolute error for Train PCI is {round(np.mean(meantrainPCI)*100,ndigits=1)}%")

mean absolute error for train Riding is 10.7%
mean absolute error for train cracking is 1.5%
mean absolute error for train rutting is 32.3%
mean absolute error for Train PCI is 5.2%


In [ ]:
Training = {'ActualTrainingRiding':train_riding_act,
            'PredictedTrainingRiding':train_riding_predict,
            'MAETrainingRiding':meantrainriding,
            'ActualTrainingCracking':train_cracking_act,
            'PredictedTrainingCracking':train_cracking_predict,
            'MAETrainingCracking':meantraincracking,
            'ActualTrainingRutting':train_rutting_act,
            'PredictedTrainingRutting':train_rutting_predict,
            'MAETrainingRutting':meantrainrutting,
            'ActualTriningPCI':Train_PCI_act,
            'PredictedTrainingPCI':Train_PCI_predict,
            'MAETrainingPCI':meantrainPCI}

In [ ]:
train = pd.DataFrame(Training)

In [ ]:
with pd.ExcelWriter('IowaPredtictionModel.xlsx') as writer:
    train.to_excel(writer,sheet_name='Train',index=False)